In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
import random
import pickle
from sklearn.preprocessing import StandardScaler
import os
import tensorflow as tf
from tqdm.notebook import tqdm

2023-06-06 09:02:32.309565: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-06 09:02:32.311888: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 09:02:32.352417: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 09:02:32.353255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 09:02:33.360401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
alogp_bottleneck = np.load("..//Data/small_mol_phase_3_features_for_both.npy")

In [3]:
y_valid = pd.read_csv("..//Data/smiles_with_rdkit_with_small_phase_3_outputs.csv")

In [4]:
std_targets = pickle.load( open('..//Data//target_scaler.pkl', 'rb'))

/home/statgrads/vpiyush2/.conda/envs/enkf/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
y_train = std_targets.transform(y_valid.iloc[:,1:])

In [6]:
targets_train = y_train

In [7]:
targets_train.shape

(959, 2)

In [8]:
targets_train.var(0)

array([2.03295671, 1.57935602])

In [9]:
np.sqrt(((targets_train - targets_train.mean(0))**2).mean(0))

array([1.42581791, 1.25672432])

In [10]:
images_train = alogp_bottleneck[:, :32]

In [11]:
images_train.shape

(959, 32)

In [12]:
doc2vec_train = alogp_bottleneck[:, 32:]

In [13]:
def ann(hidden = 32, input_shape = 256, output_shape = 2): 
    input_layer = tf.keras.layers.Input(shape = (input_shape))
    hidden_layer = tf.keras.layers.Dense(hidden)
    hidden_output = hidden_layer(input_layer)
    pred_layer = tf.keras.layers.Dense(output_shape, activation = "relu")
    pred_output = pred_layer(hidden_output)
#     pred_output = tf.keras.layers.Activation("softmax")(pred_output)
    model = tf.keras.models.Model(input_layer, pred_output)
    return model

In [14]:
h1, h2 = 16, 16

In [15]:
ann_15 = ann(h1, 32, 2)

2023-06-06 09:02:34.755108: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-06 09:02:34.755141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: c3101.swan.hcc.unl.edu
2023-06-06 09:02:34.755148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: c3101.swan.hcc.unl.edu
2023-06-06 09:02:34.755237: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.85.5
2023-06-06 09:02:34.755255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.85.5
2023-06-06 09:02:34.755260: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.85.5


In [16]:
ann_20 = ann(h2, 32, 2)

In [17]:
total_weights_1 = ann_15.count_params()

In [18]:
total_weights_2 = ann_20.count_params()

In [19]:
total_weights = total_weights_1 + total_weights_2 + 1 

In [20]:
total_weights

1125

In [21]:
targets_train.T

array([[ 2.40047573e-03,  3.83895560e-01,  2.24939275e-01, ...,
        -2.19649480e+00,  2.62517918e+00, -5.48647979e-01],
       [-3.21179953e-01, -2.84568313e-01, -1.20750605e+00, ...,
         2.49544534e-01, -5.07482222e-01,  1.17155539e+00]])

In [22]:
from statistics import covariance

In [23]:
covariance(targets_train[:,0], targets_train[:,1])

-0.6987741563528856

In [24]:
R_t_small_train = np.cov(targets_train.T)
# R_t_small_resid = np.cov(resids.T)

In [25]:
R_t_small_train

array([[ 2.0350788 , -0.69877416],
       [-0.69877416,  1.58100462]])

In [26]:
# lambda_val = 1

In [27]:
target_dim = 2

In [28]:
R_t_small_train

array([[ 2.0350788 , -0.69877416],
       [-0.69877416,  1.58100462]])

In [29]:
R_t_small_train = [[R_t_small_train[0,0], 0, R_t_small_train[0,1], 0],
                   [0, R_t_small_train[0,0], 0, R_t_small_train[1,0]], 
                  [R_t_small_train[0,1], 0, R_t_small_train[1,1],0], 
                  [0, R_t_small_train[0,1], 0, R_t_small_train[1,1]]]

In [30]:
R_t_small_train = np.array(R_t_small_train)

In [56]:
R_t_small_train

array([[ 2.0350788 ,  0.        , -0.69877416,  0.        ],
       [ 0.        ,  2.0350788 ,  0.        , -0.69877416],
       [-0.69877416,  0.        ,  1.58100462,  0.        ],
       [ 0.        , -0.69877416,  0.        ,  1.58100462]])

In [57]:
var1 = R_t_small_train[0,0]
cov = R_t_small_train[0,1]
var2 = R_t_small_train[1,1]

In [31]:
import matplotlib.pyplot as plt

In [32]:
import numpy as np

In [33]:
from scipy.stats import multivariate_normal as mvn

In [34]:
from scipy.stats import gamma

In [35]:
target_dim = 2

In [36]:
def expit(x):
    """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
    return 1 / (1 + np.exp(-x))

In [37]:
from datetime import datetime

In [38]:
from scipy.stats import beta

In [39]:
def get_targets_with_weights(batch_data, batch_data1, initial_ensembles, size_ens): 

    weights_ann_1 = ann_15.get_weights()
    weights_ann_2 = ann_20.get_weights()
    
    n_hidden_1 = len(weights_ann_1[0].ravel())
    
    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)
    
    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)


    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, target_dim)


    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + target_dim)].reshape(size_ens, 1, target_dim)


    final_output_1 = output_1 + output_layer_bias_1



    n_hidden_2 = len(weights_ann_2[0].ravel())

    initial_ensembles_1 = initial_ensembles.copy()[:, total_weights_1:(total_weights_1+ total_weights_2)]

    hidden_weights_2 = initial_ensembles_1[:,:n_hidden_2].reshape(size_ens, batch_data1.shape[1], h2)



    hidden_output_2 = np.einsum('ij,kjl->kil', batch_data1, hidden_weights_2)

    hidden_layer_bias_2 = initial_ensembles_1[:,n_hidden_2:(n_hidden_2 + h2)].reshape(size_ens, 1,  h2)


    # hidden_layer_bias_2 = np.expand_dims(hidden_layer_bias_2, 0)

    hidden_output_2 = hidden_output_2+ hidden_layer_bias_2
#     hidden_layer_bias_2 = hidden_layer_bias_2 + hidden_layer_bias_2


    n_pred_weights_2 = len(weights_ann_2[2].ravel())

    output_weights_2 = initial_ensembles_1[:,(n_hidden_2 + h2):(n_hidden_2 + h2 + n_pred_weights_2) ].reshape(size_ens, h2, target_dim)


    output_2 = np.einsum('ijk,ikl->ijl', hidden_output_2, output_weights_2)


    output_layer_bias_2 = initial_ensembles_1[:,(n_hidden_2 + h2 + n_pred_weights_2):(n_hidden_2 + h2 + n_pred_weights_2 + target_dim)].reshape(size_ens, 1, target_dim)

    # output_layer_bias_2 = np.expand_dims(output_layer_bias_2, 0)


    final_output_2 = output_2 + output_layer_bias_2


    weights_1 = initial_ensembles[:, :total_weights_1]

    weights_2 = initial_ensembles[:, total_weights_1:(total_weights_1 + total_weights_2)]

#     print(weights_1.shape)
    
    # print(weights_2.shape)

    avg_weights = initial_ensembles[:, -1].reshape(-1,1)
    
    # avg_weights_not_sig = avg_weights.reshape(avg_weights.shape[0], 1, avg_weights.shape[1])
    
    
    avg_weights_sig = expit(avg_weights)
    
    avg_weights_sig = avg_weights_sig.reshape(avg_weights_sig.shape[0], 1, avg_weights_sig.shape[1])
    
    
    complement_weights_sig = 1 - expit(avg_weights)
    
    complement_weights_sig = complement_weights_sig.reshape(complement_weights_sig.shape[0], 1, complement_weights_sig.shape[1])
    

    final_output_1 = final_output_1*complement_weights_sig
    
    final_output_2 = final_output_2*avg_weights_sig
    
    final_output_1 = final_output_1.reshape(final_output_1.shape[0], final_output_1.shape[1]*final_output_1.shape[2])
    
    final_output_2 = final_output_2.reshape(final_output_2.shape[0], final_output_2.shape[1]*final_output_2.shape[2])

    weights_1_add = np.zeros((size_ens, (total_weights_2 - total_weights_1)))



    weights_1 = np.hstack((weights_1, weights_1_add))
    
    stack_1 = np.concatenate((final_output_1, weights_1 , np.zeros((size_ens,1))), axis = 1)

    stack_2 = np.concatenate((final_output_2, weights_2 , avg_weights), axis = 1)
    
    initial_aug_state = np.hstack((stack_1, stack_2))
    
    return initial_aug_state, final_output_1, final_output_2

In [40]:
from scipy.linalg import block_diag

In [41]:
import random

In [42]:
import pickle

In [43]:
total_weights

1125

In [44]:
total_weights//4

281

In [45]:
from sklearn.metrics import mean_squared_error

In [52]:
targets_train.shape

(959, 2)

In [82]:
def rep_one(idx, batch_size = 32, var_weights = 0.5, reduction = 4,inflation_factor = 0.01, 
           R_t_small = R_t_small_train, fudge = 1): 
    
    size_ens = int(total_weights//reduction)
    for_convex = beta(a = 1, b = 1).rvs(size_ens)
#     print(for_convex.mean())
    for_convex = np.log(for_convex/(1-for_convex)).reshape(-1,1)
    
    X_train_logits = targets_train
    train_idx = list(range(0, X_train_logits.shape[0]))
    batch_chunks = [train_idx[i:i+batch_size] for i in range(0,len(train_idx),batch_size)]

    ## generate some augmented variable for iteration 0
    initial_aug_state_mean = np.zeros((total_weights-1,))
    initial_aug_state_mean = initial_aug_state_mean.reshape(-1,1)

    initial_aug_state_cov = var_weights*np.identity((total_weights-1))
    

    initial_ensembles = mvn(initial_aug_state_mean.reshape(initial_aug_state_mean.shape[0],), initial_aug_state_cov).rvs(size = size_ens)

    initial_ensembles = np.hstack((initial_ensembles,for_convex))

    train_lstm = images_train

    train_doc2vec = doc2vec_train

    train_valid_lstm = train_lstm
    train_valid_doc2vec = train_doc2vec
    
    for iter1 in tqdm(range(0,40)):
        
        batch_chunks = random.sample(batch_chunks, len(batch_chunks))

        for batch_idx in tqdm(batch_chunks):

            batch_data = train_valid_lstm[batch_idx,:]
            batch_data1 = train_valid_doc2vec[batch_idx,:]

            batch_targets = X_train_logits[batch_idx,:]
            batch_targets = batch_targets.T.ravel()
            
            column_mod_2_shape = total_weights_2 + batch_targets.shape[0]*1 + 1
        
            H_t = np.hstack((np.identity(batch_targets.shape[0]), np.zeros((batch_targets.shape[0], column_mod_2_shape-batch_targets.shape[0]))))
            
            current_aug_state, column_mod_1, column_mod_2 = get_targets_with_weights(batch_data, batch_data1, initial_ensembles, size_ens)

            current_aug_state = current_aug_state.reshape(current_aug_state.shape[0], current_aug_state.shape[1])

            current_aug_state_var = np.cov(current_aug_state.T)

            G_t = np.array([[1 , 1]]).T
            
            scirpt_H_t = np.kron(G_t.T, H_t)
            
            # print("scirpt_H_t",scirpt_H_t.shape)
            
            temp1 = current_aug_state_var@scirpt_H_t.T
            
            temp2 = scirpt_H_t@current_aug_state_var@scirpt_H_t.T
            
            # print(batch_data.shape[0])
            
            n = batch_data.shape[0]
            
            ul = var1*np.identity(n)
            
            ur = cov*np.identity(n)
            
            ll = ul
            
            lr = var2*np.identity(n)
            
            upp = np.hstack((ul, ur))
            
            loww = np.hstack((ll, lr))
            
            R_t = np.vstack((upp, loww))
            
            # R_t = block_diag(*([R_t_small] * n))
            
            K_t = temp1@np.linalg.inv(temp2 + R_t) 
        
            for ensemble_idx in range(0, current_aug_state.shape[0]):
    
            
                measurement_error =  mvn(np.repeat(0,n*2), R_t, allow_singular = True).rvs(1).ravel().reshape(-1,1)
    
                target_current = batch_targets.ravel().reshape(-1,1) + measurement_error
                
                

                current_aug_state[ensemble_idx,:] = current_aug_state[ensemble_idx,:] +(K_t@(target_current -scirpt_H_t@current_aug_state[ensemble_idx,:].reshape(-1,1))).reshape(current_aug_state.shape[1],)
    

            weights_ann_1 = current_aug_state[:, batch_targets.shape[0]:(batch_targets.shape[0] + total_weights_1)]
            
            weights_ann_2 = current_aug_state[:, (batch_targets.shape[0] + total_weights_1 + 1): (batch_targets.shape[0] + total_weights_1 + 1 + total_weights_2 )]  
            
            convex_weights = current_aug_state[:, -1].reshape(-1,1)
            
            avg_betas = expit(convex_weights)
            
            # print(np.mean(avg_betas))
        
            complement = 1-avg_betas
            
#             print(weights_ann_1.shape, weights_ann_2.shape, convex_weights.shape)
            
            
            initial_ensembles = np.hstack((weights_ann_1, weights_ann_2, convex_weights))
            
            initial_ensembles_mean = initial_ensembles.mean(0)
            
            initial_ensembles_cov = np.cov(initial_ensembles.T) +fudge*np.identity(initial_ensembles.shape[1])
            
            # print(initial_ensembles_cov.shape)
            
            initial_ensembles = mvn(initial_ensembles_mean.reshape(initial_ensembles_mean.shape[0],), initial_ensembles_cov, allow_singular=True).rvs(size = size_ens)
            
            weights_ann_1 = current_aug_state[:, batch_targets.shape[0]:(batch_targets.shape[0] + total_weights_1)]
            
            weights_ann_2 = current_aug_state[:, (batch_targets.shape[0] + total_weights_1 + 1): (batch_targets.shape[0] + total_weights_1 + 1 + total_weights_2 )]  
            
            convex_weights = current_aug_state[:, -1].reshape(-1,1)    
            
            initial_ensembles = np.hstack((weights_ann_1, weights_ann_2, convex_weights))
            
#             print(initial_ensembles.shape)
            
            current_aug_state1, column_mod_11, column_mod_21 = get_targets_with_weights(train_lstm, train_doc2vec, initial_ensembles, size_ens)
            
            initial_targets = column_mod_11 + column_mod_21
            
            initial_targets = initial_targets.reshape(initial_targets.shape[0], train_lstm.shape[0], target_dim)
            
            li = np.percentile(initial_targets, axis = 0, q = (2.5, 97.5))[0,:,:]
            
            ui = np.percentile(initial_targets, axis = 0, q = (2.5, 97.5))[1,:,:]
            
            width = ui - li
            
            avg_width_train = width.ravel().mean()
            
            # interim = expit(np.vstack((catch_train_logits[idx] , catch_valid_logits[idx] )))
            
            
            ind = (targets_train >= li) & (targets_train <= ui)
            
            coverage_train= ind.ravel().mean()
            
            preds_train = initial_targets.mean(0)
            
            truth = targets_train
            
            preds= preds_train
            
        
            rmse = np.sqrt(((truth - preds)**2).mean(0))

            print("RMSE is " + str(rmse))
            # print("RMSE over is " + str(rmse_over))
            print("coverage_train is " + str(coverage_train))
            print("avg_width_train is " + str(avg_width_train))            
            

    return truth, preds, rmse_over

In [83]:
truth, preds, rmse_over = rep_one(0)

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

RMSE is [6.7180513  2.03337489]
coverage_train is 1.0
avg_width_train is 41.603021797920285
RMSE is [3.9576351  4.34860565]
coverage_train is 1.0
avg_width_train is 37.506509430721934
RMSE is [4.89589145 2.48343272]
coverage_train is 1.0
avg_width_train is 28.564117517180712
RMSE is [5.26430728 1.74679688]
coverage_train is 1.0
avg_width_train is 23.252951324812557
RMSE is [1.7368109  1.64433673]
coverage_train is 1.0
avg_width_train is 14.086913382989428
RMSE is [2.19352611 3.63985987]
coverage_train is 0.9994786235662148
avg_width_train is 9.294204931999099
RMSE is [2.05161834 1.5761133 ]
coverage_train is 0.9442127215849844
avg_width_train is 5.324823032979305
RMSE is [1.66232204 1.51034118]
coverage_train is 0.9327424400417101
avg_width_train is 4.928649702215528
RMSE is [7.45328447 3.99840239]
coverage_train is 0.9807090719499478
avg_width_train is 10.439386465454076
RMSE is [2.05903707 2.8278507 ]
coverage_train is 0.913451511991658
avg_width_train is 4.54858320807223
RMSE is [2.

  0%|          | 0/30 [00:00<?, ?it/s]

RMSE is [1.55455265 1.25696461]
coverage_train is 0.3034410844629823
avg_width_train is 0.8337787229427032
RMSE is [1.55673596 1.25694746]
coverage_train is 0.29509906152241916
avg_width_train is 0.8014006114600224
RMSE is [1.56089878 1.25703584]
coverage_train is 0.28102189781021897
avg_width_train is 0.7712328039276429
RMSE is [1.56067176 1.25696817]
coverage_train is 0.2789363920750782
avg_width_train is 0.7567750137676231
RMSE is [1.56219512 1.25697161]
coverage_train is 0.27476538060479666
avg_width_train is 0.7479307091387353
RMSE is [1.56002993 1.25690896]
coverage_train is 0.26850886339937435
avg_width_train is 0.7293816037689307
RMSE is [1.55919365 1.25691241]
coverage_train is 0.2664233576642336
avg_width_train is 0.7206325329587214


KeyboardInterrupt: 

In [ ]:
total_weights_1 + 256 + 1

In [ ]:
# truth

In [ ]:
# preds

In [ ]:
# rep_one(0)